# Introduction to PyRational ADG models

**author:Alessio Benavoli**
<a href="http://www.alessiobenavoli.com"> alessiobenavoli.com </a>

We will see in this tutorial the basics for building a <span style="color:gray">Py</span>Rational model.

We will consider  **ADG: Almost Desirable Gambles** belief model. 

To understand this model, we consider a coin tossing *experiment* whose outcome  $\omega$ belongs to the  space of possibilities $\Omega$ (Head or Tail). We can model agent's beliefs (we call our agent Alice) about $\omega$ by asking her whether she accepts engaging in certain **risky transactions**, called **gambles**, whose outcome depends on the actual
outcome of the experiment. 

Mathematically, a gamble is a bounded real-valued function on $\Omega$, $g:\Omega 
\rightarrow \mathbb{R}$. If Alice accepts a gamble $g$, this means that she commits herself to 
receive $g(\omega)$ euros if the outcome of the experiment eventually happens 
to be the event $\omega \in \Omega$. Since $g(\omega)$ can be negative, Alice can also lose euros. Therefore Alice's acceptability of a gamble depends on her knowledge about the experiment.

The  set of gambles that Alice accepts is called her set of *desirable gambles*. 
One such set is said to be **coherent** when it satisfies the following criteria:

* A.1: Any gamble $g$ such that $g(\omega)\geq0$ for each $\omega \in \Omega$ must be desirable for Alice, given that it may increase Alice's capital without ever decreasing it  (**accepting partial gain**). 
* A.2: Any gamble $g$ such that $\max g<0$  must not be desirable for Alice, given 
that it may only decrease Alice's capital without ever increasing it  (**avoiding sure loss**). 
* A.3: If Alice finds $g$  desirable,  then also $\lambda g$ must be desirable for her for any $0<\lambda \in \mathbb{R}$ (**positive homogeneity**).
* A.4: If Alice finds $g_1$ and $g_2$ desirable, then she also must accept $g_1+g_2$ (**additivity**). 
* A.5: If Alice finds $g+\delta$  desirable for every $\delta\geq0$, then also $g$ should be desirable to her (**closure**). 

If the set of desirable gamble $G$ satisfies these property we say that it is *coherent*, or, equivalently, that **Alice is rational**. 

Note how the first four axioms express some truly minimal requirements: the first means that Alice likes to increase her wealth; the second that she does not like to decrease it; the third and fourth together simply rephrase the assumption that Alice's utility scale is linear. The last axiom is a continuity property. 

We will now build our first ADG model. We will first import the libraries we will need:

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import absolute_import 
from PyRational.models.ADG import  ADG
from PyRational.special.gambles import indicator
import numpy as np
from sympy import symbols,  FiniteSet, Piecewise, Eq

*PyRational* requires numerical input variables. We therefore encode categorical symbols (Head and Tail) into integer data (1, 0). 

*PyRational* uses *Sympy* for symbolic mathematics. We need to define in *Sympy*  a `symbol` associated to the coin toss outcome and its relative domain {0,1} (we use Sympy `FiniteSet` for the latter).

In [2]:
x  = symbols('x');
domain_x=FiniteSet(0,1)

We are now ready to build our first *PyRational* model: **ADG**.

In [38]:
ListSymbols=[x]
ListDomains=[domain_x]
model = ADG(ListSymbols,ListDomains)
model

List of Symbols
Domain
List of desirable gambles
Avoiding sure loss?


We will clarify later the meaning of "avoiding sure loss".
The set $G$, the gambles that Alice finds desirable, is empty at the moment.

What is a gamble in the coin experiment? We can model any gamble as a combination of `indicator` functions.

In [39]:
g = -0.1*indicator('Eq(x,1)') + 1*indicator('Eq(x,0)')

In this example Alice receives 1 euro if Tails and loses -0.1 euro if Heads. Note that, in Sympy, the == must be replaced by Eq(.,.). The indicator is implemented as a `Piecewise` Sympy function:

In [40]:
g

Piecewise((1, Eq(x, 0)), (0, True)) - 0.1*Piecewise((1, Eq(x, 1)), (0, True))

Therefore, you can write the same gamble as

In [41]:
1*Piecewise((1, Eq(x, 0)), (0, True)) - 0.1*Piecewise((1, Eq(x, 1)), (0, True))

Piecewise((1, Eq(x, 0)), (0, True)) - 0.1*Piecewise((1, Eq(x, 1)), (0, True))

We can evaluate this gamble as follows:

In [42]:
print('if Tails: ', g.subs(x,0),', if Heads:',g.subs(x,1))

if Tails:  1 , if Heads: -0.100000000000000


Which gambles does Alice find desirable?

Alice is obviously willing to accept any gamble $g$ that, no matter the result of the experiment,
may increase her wealth without ever decreasing it, that is with $g \geq 0$.
For instance, the following gambles:

In [43]:
g1 = 1.0*indicator('Eq(x,1)') + 1.0*indicator('Eq(x,0)')
g2 = 0.1*indicator('Eq(x,1)') + 1.0*indicator('Eq(x,0)')

Similarly.  Alice does not accept  any gamble $g$ that will surely decrease her wealth, that is with  $\max g<0$. For the other types of gambles, their desirability depends on Alice's beliefs about the coin.


Let's assume Alice thinks that the coin is slightly biased.  So she decides to accept a gamble, only if the ratio between its positive part and negative part is greater than $1.1$:

In [36]:
g3 =    1*indicator('Eq(x,1)') - 0.1*indicator('Eq(x,0)')
g4 = -0.1*indicator('Eq(x,1)') +   1*indicator('Eq(x,0)')
g5 =    2*indicator('Eq(x,1)') - 0.5*indicator('Eq(x,0)')
g6 =  0.5*indicator('Eq(x,1)') - 0.4*indicator('Eq(x,0)')
g7 = -0.4*indicator('Eq(x,1)') + 0.5*indicator('Eq(x,0)')
g8 =  1.1*indicator('Eq(x,1)') -   1*indicator('Eq(x,0)')
g9 =   -1*indicator('Eq(x,1)') + 1.1*indicator('Eq(x,0)')

We add all these gambles to `model` as follows:

In [44]:
model.add_gambleList([g1,g2,g3,g4,g5,g6,g7,g8,g9])
model

List of Symbols
Domain
List of desirable gambles
Avoiding sure loss?


Note that $G$ is a list that includes all Alice's desirable gambles. We now `build` the belief model.

In [64]:
model.buildModel()
model

List of Symbols
Domain
List of desirable gambles
Avoiding sure loss?


You can think about "model building" as a compiling phase for the belief model. 

It adds to $G$ all gambles that are implied by axioms A.1, A.3, A.4, A.5.

This means that Alice doesn't need to say that $g_1,g_2$ (in the above example) are desirable, because this is implied by A.1. Similarly, she doesn't need to say that $10*g_3$ is also desirable to her, because this is implied by A.3 and so on.
  

Given that A.1, A.3, A.4, A.5 are satisfied, to check that Alice is rational, it remains to verify that A.2 is also satisfied. We use `check_avs` for that. 
Before doing that, we will set some options for the optimiser. I will explain the meaning of these options in another notebook.

In [46]:
optimoptions={ 'method_LISP': 'Cutting_plane', 
               'SolverLP':'linprog', 
               'LP_acc_constraints':1e-300,
               'SolverNLP':'differential_evolution',
               'NLP_alpha_cut':-0.009,
               'num_support_points': 10,
               'verbose':False}

model.check_avs(options=optimoptions)
model

Belief Model avoids sure  loss


List of Symbols
Domain
List of desirable gambles
Avoiding sure loss?


So Alice is **rational** or, equivalently, her set of desirable gambles is coherent.

### Inference

ADG allows Alice to make inferences on her model.

For instance, she may want to assess if a certain gamble $f$ is desirable to her, given  
she has accepted the gambles in $G$. In other words, she wants to assess if this gamble belongs to the set `model.G`

In [47]:
f=10*g3
f_range=(None,None)
model.natural_extension(f,f_range,options=optimoptions)

True

This is True, because the desirability of $10*g3$ follows from A.3.
Here, `f_range`is the range of $f$, for the moment we can leave it indetermined. We will use it in another notebook. Similarly, we have

In [48]:
f=10*g3+g4
f_range=(None,None)
model.natural_extension(f,f_range,options=optimoptions)

True

In [49]:
f=-g1
f_range=(None,None)
model.natural_extension(f,f_range,options=optimoptions)

False

In [50]:
f=1*Piecewise((1, Eq(x, 1)), (0, True)) - 1.5*Piecewise((1, Eq(x, 0)), (0, True))
f_range=(None,None)
model.natural_extension(f,f_range,options=optimoptions)

False

Another important inference in ADG is `lower prevision`. This allows Alice to determine
her maximum buying price for a gamble: how much she should pay to buy a certain gamble.

In [51]:
lp=model.lower_prevision(f,f_range,options=optimoptions)
print(lp)

-0.30952380952380953


In other words, `lp` is the largest value such that $f-lp$ is desirable to Alice.

In [52]:
f1=f-lp
model.natural_extension(f1,f_range,options=optimoptions)

True

In [53]:
f1=f-lp-0.00000001
model.natural_extension(f1,f_range,options=optimoptions)

False

Similarly, Alice may want to determine her minimum selling price for a gamble. This is the `upper_prevision` of $f$

In [54]:
up=model.upper_prevision(f,f_range,options=optimoptions)
print(up)

-0.19047619047619044


In [20]:
f1=up-f
model.natural_extension(f1,f_range,options=optimoptions)

True

In [55]:
f1=up-f-0.00000001
model.natural_extension(f1,f_range,options=optimoptions)

False

## Updating: inference

Alice may want to evaluate how her inferences would change if she knew that the result of the coin toss will be Head.

This operation is called `updating` in ADG and is formally equivalent to probabilistic conditioning. It is performed by firstly defining an indicator on the event of interest:

In [56]:
h = Piecewise((1.0,Eq(x,1)),(0.0,Eq(x,0)))

and then re-computing `lower_prevision` and `upper_prevision` conditionally on this event.

In [58]:
lp=model.lower_prevision(f,f_range,h=h,options=optimoptions)
print(lp)

1.0


In [60]:
up=model.upper_prevision(f,f_range,h=h,options=optimoptions)
print(up)

1.0


Note that, since she knows that the result is Heads, then she is sure that the payoff for $f$ is one.

## Updating: model

Alice may also want to update her whole belief model conditionally on $h$. This can be done as follows:

In [61]:
NewG = model.updating(h,options=optimoptions)
NewG

[1.0*Piecewise((1, Eq(x, 0)), (0, True)) + 1.0*Piecewise((1, Eq(x, 1)), (0, True)) - 1.0,
 1.0*Piecewise((1, Eq(x, 0)), (0, True)) + 0.1*Piecewise((1, Eq(x, 1)), (0, True)) - 0.1,
 -0.1*Piecewise((1, Eq(x, 0)), (0, True)) + Piecewise((1, Eq(x, 1)), (0, True)) - 1.0,
 Piecewise((1, Eq(x, 0)), (0, True)) - 0.1*Piecewise((1, Eq(x, 1)), (0, True)) + 0.1,
 -0.5*Piecewise((1, Eq(x, 0)), (0, True)) + 2*Piecewise((1, Eq(x, 1)), (0, True)) - 2.0,
 -0.4*Piecewise((1, Eq(x, 0)), (0, True)) + 0.5*Piecewise((1, Eq(x, 1)), (0, True)) - 0.5,
 0.5*Piecewise((1, Eq(x, 0)), (0, True)) - 0.4*Piecewise((1, Eq(x, 1)), (0, True)) + 0.4,
 -Piecewise((1, Eq(x, 0)), (0, True)) + 1.1*Piecewise((1, Eq(x, 1)), (0, True)) - 1.1,
 1.1*Piecewise((1, Eq(x, 0)), (0, True)) - Piecewise((1, Eq(x, 1)), (0, True)) + 1.0]

It returns the updated set of desirable gambles. We can use it to build a new belief model.

In [62]:
model1 = ADG(ListSymbols,ListDomains)
model1.add_gambleList(NewG)
model1.buildModel()
model1.check_avs()
model1

Belief Model avoids sure  loss


List of Symbols
Domain
List of desirable gambles
Avoiding sure loss?


In [63]:
model1.lower_prevision(f,f_range,options=optimoptions)

1.0

This is inference is equivalent to `model1.lower_prevision(f,f_range,h=h,options=optimoptions)`